In [1]:
############
# Packages #
############
import time as t

import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import scipy as sp
import random

###################
# hyperparametres #
###################

# for reproductibility
seed = 0

# data dimensions
T=200
k=100
l =0

#Z : (k,1)
#Y : (T,1)
#U : (T,l)
#X : (T,k)

# data generations

# for X:
rho=0.75

# for Beta : number of non null
s=5 #in [5,10,100]
lst_s = [5,10,100]

# for sigma2 :ratio between explained and total variance 
Ry=0.02 #in [0.02, 0.25, 0.5]
lst_Ry = [0.02, 0.25, 0.5]

# for q prior
a=1
b=1

# for R2 prior
A=1
B=1

### Initialize parameters

Firstly, let's comput the initialization of the parameters

In [2]:
def compute_X(T, k, rho):
    """Compute matrix of xt observations

    Args:
        T (int): number of observations
        k (int): number of xt predictors
        rho (float): Toeplitz correlation parameter

    Returns:
        np.array: dimensions T*k
    """
    cov_matrix=np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            cov_matrix[i,j]=rho**np.abs(i-j)
    return np.random.multivariate_normal([0]*k, cov_matrix, T)

def compute_U(T, l):
    """Compute matrix of ut observations

    Args:
        T (int): number of observations
        l (int): number of ut predictors

    Returns:
        int: 0
    """
    if l==0:
        return 0
    else:
        pass
        
def compute_vx(X):
    """Compute mean estimated variance of xt predictors

    Args:
        X (np.array): matrix of xt predictors

    Returns:
        float: mean estimated variance of xt predictors
    """
    return np.mean(np.var(X,axis=0))

def sample_beta(k, s):
    """Sample of beta vector of dimensions 1*k

    Args:
        k (int): number of predictors
        s (int): number of non-zero elements of beta

    Returns:
        np.array: dimensions 1*k
    """
    beta=np.zeros(k)
    index_normal_distribution=np.random.choice(len(beta), size=s, replace=False)
    beta[index_normal_distribution] = np.random.normal(loc=0, scale=1, size=s)
    return beta

def sample_phi(l):
    """Sample phi prior

    Args:
        l (int): number of ut predictors

    Returns:
        int or np.array: phi samples
    """
    if l==0:
        return 0
    else:
        return np.random.uniform(0,1, size=l)

def compute_Z(beta):
    """Compute z_1,...,z_k

    Args:
        beta (np.array): random vector beta

    Returns:
        np.array: dimensions1*k
    """
    Z=beta
    Z[Z!=0]=1
    return Z

def compute_sigma2(Ry, beta, X):
    """ Compute sigma2
    Args:
        Ry (float): pourcentage of explained variance
        beta (np.array): random vector beta
        X (np.array): matrix of xt predictors

    Returns:
        float: dimensions 1*1
    """
    return (1/Ry-1)*np.mean(np.square(X @ beta))

def sample_R2(A,B):
    """Sample R^2 according to a beta distribution

    Args:
        A (float): shape parameter
        B (float): shape parameter

    Returns:
        float: R^2 random variable
    """
    return np.random.beta(A,B)

def sample_q(a,b):
    """Sample q according to a beta distribution

    Args:
        a (float): shape parameter
        b (float): shape parameter

    Returns:
        float: q random variable
    """
    return np.random.beta(a,b)

def compute_gamma2(R2, q, k, vx):
    """Compute gamma^2 by inverting the R^2 function

    Args:
        R2 (float): R^2 random variable
        q (float): q random variable
        k (int): number of xt predictors
        vx (float): mean estimated variance of xt predictors

    Returns:
        float: gamma^2 random variable
    """
    return R2/((1-R2)*q*k*vx)

def sample_epsilon(T, sigma2):
    """Sample epsilon_1,...,epsilon_T

    Args:
        T (int): number of observations
        sigma2 (float): sigma2 previously sampled

    Returns:
        np.array: dimensions 1*T
    """
    return np.random.normal(loc=0, scale=sigma2, size=T)

def compute_Y(X, beta, epsilon):
    """Compute y_1,...,y_T

    Args:
        X (np.array): matrix of xt predictors
        beta (np.array): random vector beta
        epsilon (np.array): vector of epsilon_1,...,epsilon_T

    Returns:
        np.array: dimensions 1*T
    """
    return X@beta + epsilon


### Final function
def init_parameters(seed, T, k, l, rho, s, Ry, a, b, A, B):
    """
    Initialize parameters for a given simulation.

    Args:
        seed (int): Seed for reproducibility.
        T (int): Number of observations.
        k (int): Number of covariates.
        l (int): Number of latent variables.
        rho (float): Correlation parameter.
        s (float): Scaling parameter.
        Ry (float): Response variance.
        a (float): Shape parameter for gamma2.
        b (float): Shape parameter for gamma2.
        A (float): Shape parameter for q.
        B (float): Shape parameter for q.

    Returns:
        dict: Dictionary containing initialized parameters.
    """
    np.random.seed(seed=seed)
    dct = {
        "X" : compute_X(T=T, k=k, rho=rho),
        "U": compute_U(T=T, l=l),
        "beta": sample_beta(k=k, s=s),
        "phi": sample_phi(l=l),
        "q": sample_q(a,b)
    }
    dct["R2"] = sample_R2(A,B)
    dct["gamma2"]=compute_gamma2(R2=dct["R2"], q=dct["q"], k=k, vx=compute_vx(dct["X"]))
    dct["Z"]=compute_Z(beta=dct["beta"])
    dct["sigma2"] = compute_sigma2(Ry=Ry, beta=dct["beta"], X=dct["X"])
    dct["epsilon"] = sample_epsilon(T=T, sigma2=dct["sigma2"])
    dct["Y"]=compute_Y(X=dct["X"], beta=dct["beta"], epsilon=dct["epsilon"])
    return dct

In [3]:
dct = init_parameters(seed,T,k,l,rho,s,Ry,a,b,A,B)

Z=dct['Z']
X=dct['X']
U=dct['U']
Y=dct['Y']
q=dct["q"]
gamma2=dct["gamma2"]
beta=dct["beta"]
phi=dct["phi"]
sigma2=dct["sigma2"]
R2=dct["R2"]


## 1. Draw from the conditional posterior of (R2, q)

In [4]:
def sample_discrete(seed, values, weights, n_points):
    probs = weights/weights.sum()
    return sp.stats.rv_discrete(seed = seed, values = (values, probs)).rvs(size=n_points)

def sample_discrete_ndim(seed, values, weights, n_points):
    nval = np.multiply(values.shape)
    return sample_discrete(seed, values.reshape(nval), weights.reshape(nval), n_points)

def density_unormalized_R2_q_by_Y_U_X_theta_z(R2, q, X, Z, sigma2, k, beta, a, b, A, B):
    s_z = np.sum(Z)
    vx=compute_vx(X)
    exponent = - np.prod([
        1/(1e-6 + 2*sigma2),
        (k*vx*q*(1-R2))/(1e-6 + R2),
         np.dot(beta, np.dot(np.diag(Z), beta))
        ])       
    return np.prod([
        np.exp(exponent),
        q**(s_z+s_z/2+a-1),
        (1-q)**(k-s_z+b-1),
        R2**(A-1-s_z/2),
        (1-R2)**(s_z/2+B-1)
    ])

def sample_R2_q_by_Y_U_X_theta_z(X, Z, sigma2, beta, a, b, A, B, n_variables, seed):
    k=X.shape[1]
    arr0 = np.arange(0.001,0.101,0.001) # commence pas à 0 car division par 0 sinon
    arr1 = np.arange(0.11,0.91,0.01)
    arr2 = np.arange(0.901,1.001,0.001)
    discretization = np.concatenate((arr0, arr1, arr2), axis=0)
    
    values = np.dstack(np.meshgrid(discretization, discretization)).reshape(-1, 2)
    
    def density(R2_q):
        R2 = R2_q[0]
        q = R2_q[1]
        return density_unormalized_R2_q_by_Y_U_X_theta_z(R2, q, X, Z, sigma2, k, beta, a, b, A, B)
    
    weights = np.apply_along_axis(density, 1, values) 
    index = np.arange(len(weights))
    sample_mask = sample_discrete(seed, index, weights, n_variables)
    return values[sample_mask]

In [5]:
posterior_R2_q=sample_R2_q_by_Y_U_X_theta_z(X=X, Z=Z, sigma2=sigma2, beta=beta, a=a, b=b, A=A, B=B, n_variables=1, seed=seed)
print(posterior_R2_q)

[[0.028 0.063]]


### Yanis: test pour simuler le posterior de (R2,q) par MH random walk

In [6]:
def unormalized_posterior_density_R2_q(R2, q, Y, U, X, sigma2, phi, beta, Z, a, b, A, B):
    s_z = np.sum(Z)
    vx=compute_vx(X)
    k=X.shape[1]
    
    return np.exp(1/(-2*sigma2)*((k*vx*q*(1-R2)/R2))*beta.T@np.diag(Z)@beta)*q**(s_z+0.5*s_z+a-1)*(1-q)**(k-s_z+b-1)*R2**(A-1-s_z*0.5)*(1-R2)**(0.5*s_z+B-1)

In [7]:
def MH_RW_posterior_q_R2(R2, q, proposal_std, Y, U, X, sigma2, phi, beta, Z, a, b, A, B, seed):
    np.random.seed(seed)
    current_R2=R2
    current_q=q

    proposed_R2 = current_R2 + np.random.normal(0, proposal_std)
    proposed_q = current_q + np.random.normal(0, proposal_std)

    current_density = unormalized_posterior_density_R2_q(current_R2, current_q, Y, U, X, sigma2, phi, beta, Z, a, b, A, B)
    proposed_density = unormalized_posterior_density_R2_q(proposed_R2, proposed_q, Y, U, X, sigma2, phi, beta, Z, a, b, A, B)

    acceptance_ratio = min(1, proposed_density / current_density)
    u=np.random.uniform(0,1,1)[0]

    while u>acceptance_ratio:
        current_R2=proposed_R2
        current_q=proposed_q

        proposed_R2 = current_R2 + np.random.normal(0, proposal_std)
        proposed_q = current_q + np.random.normal(0, proposal_std)

        current_density = unormalized_posterior_density_R2_q(current_R2, current_q, Y, U, X, sigma2, phi, beta, Z, a, b, A, B)
        proposed_density = unormalized_posterior_density_R2_q(proposed_R2, proposed_q, Y, U, X, sigma2, phi, beta, Z, a, b, A, B)

        acceptance_ratio = min(1, proposed_density / current_density)
        u=np.random.uniform(0,1,1)[0]
    else:
        return proposed_R2, proposed_q

In [8]:
posterior_R2, posterior_q = MH_RW_posterior_q_R2(R2=R2, q=q, proposal_std=0.1, Y=Y, U=U, X=X, sigma2=sigma2, phi=phi, beta=beta, Z=Z, a=a, b=b, A=A, B=B, seed=0)
posterior_R2_q=[list([posterior_R2, posterior_q])]

## 2. Sample from the conditional posterior of $\phi$

In [9]:
def sample_phi_posterior(Y, U, X, beta, sigma2, n_variables, seed):
    """Sample 1*l random vectors ϕ|Y, U, X, z, β, R^2, q, sigma^2

    Args:
        Y (np.array): T*1 vector of target
        U (np.array): T*l matrix of predictors
        X (np.array): T*k matrix of predictors
        beta (np.array): 1*k vector beta
        sigma2 (float): sigma^2
        n_variables (int): number of variables
        seed (int): seed

    Returns:
        np.array: n_variables of 1*l random vectors
    """
    np.random.seed(seed)
    if isinstance(U, np.ndarray):
        return np.random.multivariate_normal(np.linalg.inv(U.T@U)@U.T@(Y-X@beta), sigma2*np.linalg.inv(U.T@U), n_variables)
    else:
        l=0
        return [sample_phi(l=l)]*n_variables


In [10]:
phi_posterior=sample_phi_posterior(Y=Y, U=U, X=X, beta=beta, sigma2=sigma2, n_variables=2, seed=seed)
print(phi_posterior)

[0, 0]


## 3. Sample from the conditional posterior of $z$

### Preliminary step: compute some random vectors

In [11]:
def compute_X_tilde(X,Z):
    """Compute T*s(z) \tilde{X} matrix

    Args:
        X (np.array): T*p matrix of x predictors
        Z (np.array): 1*k vector of z

    Returns:
        np.array: \tilde{X} matrix
    """
    non_zero_z=list(np.nonzero(Z)[0])
    X_tilde=X[:, non_zero_z]
    return X_tilde

def compute_W_tilde(X_tilde, gamma2):
    """Compute s(z)*s(z) \tilde{W} matrix

    Args:
        X_tilde (np.array): T*s(z) \tilde{X} matrix
        gamma2 (float): gamma^2 

    Returns:
        np.array: \tilde{W} matrix
    """
    I_s_z=np.identity(X_tilde.shape[1])
    return X_tilde.T@X_tilde+(1/gamma2)*I_s_z

def compute_Y_tilde(Y, U, phi):
    """Compute 1*T \Tilde{Y} matrix

    Args:
        Y (np.array): 1*T vector of target variables
        U (np.array): T*l matrix of u predictors
        phi (np.array): 1*l phi vector

    Returns:
        np.array: \Tilde{Y}
    """
    if U==0:
        return Y
    else:
        return Y-U@phi

def compute_estimator_beta_tilde(W_tilde, X_tilde, Y_tilde):
    """Compute 1*s(z) \hat{\tilde{\beta}} vector

    Args:
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        X_tilde (np.array): T*s(z) \Tilde{X} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix

    Returns:
        np.array: 1*s(z) \hat{\Tilde{\beta}} vector
    """
    return np.linalg.inv(W_tilde)@X_tilde.T@Y_tilde

In [12]:
X_tilde=compute_X_tilde(X,Z)
W_tilde=compute_W_tilde(X_tilde, gamma2)
Y_tilde=compute_Y_tilde(Y, U, phi)
est_beta_tilde=compute_estimator_beta_tilde(W_tilde, X_tilde, Y_tilde)

### Gibbs sampler

In [13]:
def compute_z_posterior(Z, Y, U, X, phi, R2, q):
    """Compute \pi(z|Y, U, X, \phi, R^2, q) mass function

    Args:
        Z (np.array): 1*k array of z1,...,zk
        Y (np.array): 1*T vector of target variables
        U (np.array): T*l matrix of u predictors
        X (np.array): T*p matrix of x predictors
        phi (np.array): 1*l phi vector
        R2 (float): R^2 random variable
        q (float): q variable


    Returns:
        float: value of \pi(z|Y, U, X, \phi, R^2, q)
        
    """
    T=len(Y)
    vx=compute_vx(X)
    X_tilde=compute_X_tilde(X, Z)
    gamma2=compute_gamma2(R2, q, k, vx)
    W_tilde=compute_W_tilde(X_tilde, gamma2)
    Y_tilde=compute_Y_tilde(Y, U, phi)
    estimator_beta_tilde=compute_estimator_beta_tilde(W_tilde, X_tilde, Y_tilde)
    
    s_z=np.sum(Z)
    return q**s_z*(1-q)**(k-s_z)*(1/gamma2)**(s_z/2)*np.linalg.det(W_tilde)**(-0.5)*0.5**(-T/2)*(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@W_tilde@estimator_beta_tilde)**(-T/2)*sp.special.gamma(T/2)


def compute_zi_posterior_conditional_z_excluded_i(z_i, z_minus_i, i, q, Y, U, X, phi, R2):
    """Compute \pi(z_i|Y, X, U, \phi, R^2, q, z_{-i}) for a specific zi

    Args:
        z_i (int): z_i variable
        z_minus_i (np.array): z random vector with z_i excluded
        i (int): index of z_i variable in z random vector
        q (float): q variable
        Y (np.array): 1*T vector of target variables
        U (np.array): T*l matrix of u predictors
        X (np.array): T*p matrix of x predictors
        phi (np.array): 1*l phi vector
        R2 (float): R^2 random variable

    Returns:
        float: value of \pi(z_i|Y, X, U, \phi, R^2, q, z_{-i})
    """

    z=z_minus_i
    z=np.insert(z, i, z_i)
    z_posterior=compute_z_posterior(Z=z, Y=Y, U=U, X=X, phi=phi, R2=R2, q=q)
    z_0=np.copy(z)
    z_0[i]=0
    z_posterior_zi_equal_0=compute_z_posterior(Z=z_0, Y=Y, U=U, X=X, phi=phi, R2=R2, q=q)
    z_1=np.copy(Z)
    z_1[i]=1
    z_posterior_zi_equal_1=compute_z_posterior(Z=z_1, Y=Y, U=U, X=X, phi=phi, R2=R2, q=q)

    return z_posterior/(z_posterior_zi_equal_0+z_posterior_zi_equal_1+1e-6)

def simulated_zi_posterior_conditional_z_excluded_i(z_minus_i, i, q, Y, U, X, phi, R2):
    """Sample z_i|Y, X, U, \phi, R^2, q z_{-i}

    Args:
        z_minus_i (np.array): z random vector with z_i excluded
        i (int): index of z_i variable in z random vector
        q (float): q variable
        Y (np.array): 1*T vector of target variables
        U (np.array): T*l matrix of u predictors
        X (np.array): T*p matrix of x predictors
        phi (np.array): 1*l phi vector
        R2 (float): R^2 random variable

    Returns:
        int: value of z_i|Y, X, U, \phi, R^2, q z_{-i} in {0,1} 
    """
    proba_success=compute_zi_posterior_conditional_z_excluded_i(z_i=1, z_minus_i=z_minus_i, i=i, q=q, Y=Y, U=U, X=X, phi=phi, R2=R2)
    return np.random.binomial(n=1, p=proba_success, size=1)[0]
    # u=np.random.uniform(0,1)
    # if u<=1-proba_success:
    #     return 0 
    # else:
    #     return 1
    
def step_gibbs_sampler_z_posterior(q, Y, U, X, phi, R2, z_t_minus_1):
    """One iteration of Gibbs sampler

    Args:
        q (float): q variable
        Y (np.array): 1*T vector of target variables
        U (np.array): T*l matrix of u predictors
        X (np.array): T*p matrix of x predictors
        phi (np.array): 1*l phi vector
        R2 (float): R^2 random variable
        z_t_minus_1 (_type_): z_{t-1} variable sampled during the previous iteration (t-1 step)

    Returns:
        np.array: array of variables Z^(t)=(Z^(t)_1,...,Z^(t)_k) sampled at step t
    """
    z_t=np.copy(z_t_minus_1)
    for i in range(len(z_t)):
        z_t_minus_i=np.delete(z_t, i)
        sampled_z_t_i=simulated_zi_posterior_conditional_z_excluded_i(z_minus_i=z_t_minus_i, i=i, q=q, Y=Y, U=U, X=X, phi=phi, R2=R2)
        z_t=np.insert(z_t_minus_i, i, sampled_z_t_i)
    return z_t

def gibbs_sampler_z_posterior(q, Y, U, X, phi, R2, n_iter, n_variables, seed):
    """Gibbs sampler to simulate 1*k vectors z|Y, U, X, \phi, R^2

    Args:
        q (float): q variable
        Y (np.array): 1*T vector of target variables
        U (np.array): T*l matrix of u predictors
        X (np.array): T*p matrix of x predictors
        phi (np.array): 1*l phi vector
        R2 (float): R^2 random variable
        n_iter (int): number of iterations
        n_variables (int): number of variables desired
        seed (int): seed

    Returns:
        np.array: array of n_variables z=(z_1,...,z_k)
    """
    np.random.seed(seed)
    array_z=[]
    for n in range(n_variables):
        z_0=np.random.binomial(n=1, p=q, size=k) #Yanis; je ne sais pas trop comment initialiser le premier Z, j'ai repris la loi a priori
        z_t=z_0
        for t in range(n_iter):
            z_t=step_gibbs_sampler_z_posterior(q=q, Y=Y, U=U, X=X, phi=phi, R2=R2, z_t_minus_1=z_t)
        array_z.append(z_t)
    return np.array(array_z)

In [14]:
# T=len(Y)
# Z=np.array([random.choice([0, 1]) for _ in range(k)])

# vx=compute_vx(X)
# X_tilde=compute_X_tilde(X, Z)
# gamma2=compute_gamma2(R2, q, k, vx)
# W_tilde=compute_W_tilde(X_tilde, gamma2)
# Y_tilde=compute_Y_tilde(Y, U, phi)
# estimator_beta_tilde=compute_estimator_beta_tilde(W_tilde, X_tilde, Y_tilde)
    
# s_z=np.sum(Z)

# q**s_z*(1-q)**(k-s_z)*(1/gamma2)**(s_z/2)*np.linalg.det(W_tilde)**(-0.5)*0.5**(-T/2)*(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@W_tilde@estimator_beta_tilde)**(-T/2)*sp.special.gamma(T/2)

In [15]:
sample_posterior_z=gibbs_sampler_z_posterior(q=q,
                                             Y=Y,
                                             U=U,
                                             X=X,
                                             phi=phi,
                                             R2=R2,
                                             n_iter=1000,
                                             n_variables=1,
                                             seed=seed)

print(sample_posterior_z)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [16]:
# import random

# for t in range(10000):
#     z_test=np.array([random.choice([0, 1]) for _ in range(k)])
#     if compute_z_posterior(z_test, Y, U, X, phi, R2, q)!=0:
#         print(z_test)

### 4. Draw from the conditional posterior of $\sigma^2$

In [17]:
def sample_sigma2_posterior(Y, U, X, phi, R2, q, Z, n_variables, seed):
    """Sample random variables sigma^2|Y, U, X, ϕ, R2, q, z

    Args:
        Y (np.array): 1*T vector of target variables
        U (np.array): T*l matrix of u predictors
        X (np.array): T*p matrix of x predictors
        phi (np.array): 1*l phi vector
        R2 (float): R^2 random variable
        q (float): q variable
        Z (np.array): 1*k array of z1,...,zk
        n_variables (int): number of variables desired
        seed (int): seed

    Returns:
        np.array: n_variables sigma^2|Y, U, X, ϕ, R2, q, z
    """
    np.random.seed(seed)
    T=len(Y)
    k=X.shape[1]
    X_tilde=compute_X_tilde(X,Z)
    Y_tilde=compute_Y_tilde(Y, U, phi)
    gamma2=compute_gamma2(R2, q, k, compute_vx(X))
    W_tilde=compute_W_tilde(X_tilde, gamma2)
    estimator_beta_tilde=compute_estimator_beta_tilde(W_tilde, X_tilde, Y_tilde)
    I_s_z=np.identity(X_tilde.shape[1])
    inverse_gamma_dist = sp.stats.invgamma(T/2, scale=0.5*(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@(X_tilde.T@X_tilde+(1/gamma2)*I_s_z)@estimator_beta_tilde))
    return inverse_gamma_dist.rvs(size=n_variables)

In [18]:
array_sigma2_posterior=sample_sigma2_posterior(Y=Y,
                                               U=U,
                                               X=X,
                                               phi=phi,
                                               R2=R2,
                                               q=q,
                                               Z=Z,
                                               n_variables=2,
                                               seed=seed)
print(array_sigma2_posterior)

[81820.77526225 85601.92051202]


### 5. Draw from the conditional posterior of $\tilde{\beta}$

In [19]:
def sample_beta_tilde_posterior(Y, U, X, phi, R2, q, sigma2, Z, n_variables, seed):
    """Sample 1*s(z) random vectors \tilde{β}|Y, U, X, ϕ, R2, q, sigma^2, z

    Args:
        Y (np.array): 1*T vector of target variables
        U (np.array): T*l matrix of u predictors
        X (np.array): T*p matrix of x predictors
        phi (np.array): 1*l phi vector
        R2 (float): R^2 random variable
        q (float): q variable
        sigma2 (float): sigma^2
        Z (np.array): 1*k array of z1,...,zk
        n_variables (int): number of variables
        seed (int): seed

    Returns:
        np.array: n_variables of 1*l random vectors
    """
    np.random.seed(seed)
    X_tilde=compute_X_tilde(X,Z)
    gamma2=compute_gamma2(R2, q, k, compute_vx(X))
    I_s_z=np.identity(X_tilde.shape[1])
    if U==0:
        return np.random.multivariate_normal(np.linalg.inv((1/gamma2)*I_s_z+X_tilde.T@X_tilde)@X_tilde.T@Y,sigma2*np.linalg.inv((1/gamma2)*I_s_z+X_tilde.T@X_tilde), n_variables)
    else:
        return np.random.multivariate_normal(np.linalg.inv((1/gamma2)*I_s_z+X_tilde.T@X_tilde)@X_tilde.T@(Y-U@phi),sigma2*np.linalg.inv((1/gamma2)*I_s_z+X_tilde.T@X_tilde), n_variables)

In [20]:
array_beta_tilde_posterior=sample_beta_tilde_posterior(Y=Y,
                                                       U=U,
                                                       X=X,
                                                       phi=phi,
                                                       R2=R2,
                                                       q=q,
                                                       sigma2=sigma2,
                                                       Z=Z, 
                                                       n_variables=2,
                                                       seed=seed)

print(array_beta_tilde_posterior)

[[ -4.27628282 -10.40671818  -1.31912972  -5.18962238  -7.61567135]
 [ -4.01930256  -8.18021893  -4.92844241  -2.8766446   -5.44461419]]


## Sample from $(R^2, q, \theta)$ posterior (Gibbs sampler)

In [21]:
def reconstruct_beta_from_beta_tilde_z(beta_tilde, z, k):
    """Find \beta from \tilde{\beta} and vector z

    Args:
        beta_tilde (np.array): \tilde{beta} vector
        z (np.array): 1*k array of z1,...,zk
        k (int): number of xt predictors

    Returns:
        np.array: beta vector
    """
    beta=np.zeros(k)
    beta[z.nonzero()] = beta_tilde

    return beta

print(reconstruct_beta_from_beta_tilde_z(beta_tilde=np.array([1,2,3]), z=np.array([1,0,1,0,1]), k=5))

[1. 0. 2. 0. 3.]


In [22]:
def gibbs_sampler_theta_posterior(X, Y, U, a, b, A, B, n_iter, n_iter_gibbs_sampler_posterior_z, burn_in_period, n_variables, seed):

    np.random.seed(seed)

    dico_R2_q_theta_posterior={}
    k=X.shape[1]

    for n in range(n_variables):
        list_r2_q_theta_posterior_var_n=[] #List of accumulated posterior through the iterators for the n-th posterior we want to sample

        #Init q, R2, phi, z, sigma2, beta_tilde
        q_init=sample_q(a,b)
        R2_init=sample_R2(A,B)
        phi_init=sample_phi(l)
        z_init=compute_Z(beta)
        sigma2_init=compute_sigma2(Ry, beta, X)
        beta_tilde_init=np.copy(beta)
        beta_tilde_init=beta_tilde_init[beta_tilde_init!=0]

        #(R2, q, theta)_0=[R2_init, q_init, phi_init, beta_tilde_init, sigma2_init]
        #t=0
        posterior_q_t=q_init #(I)
        posterior_R2_t=R2_init #(I)
        posterior_phi_t=phi_init #(II)
        posterior_z_t=z_init #(III)
        posterior_sigma2_t=sigma2_init #(IV)
        posterior_beta_tilde_t=beta_tilde_init #(V)
        posterior_beta_t=reconstruct_beta_from_beta_tilde_z(beta_tilde=posterior_beta_tilde_t, z=posterior_z_t, k=k)

        for t in range(n_iter):
            posterior_R2_q_t=sample_R2_q_by_Y_U_X_theta_z(X=X,
                                                          Z=posterior_z_t,
                                                          sigma2=posterior_sigma2_t,
                                                          beta=posterior_beta_t,
                                                          a=a, b=b, A=A, B=B,
                                                          n_variables=1,
                                                          seed=None)
            posterior_R2_t=posterior_R2_q_t[0][0]
            posterior_q_t=posterior_R2_q_t[0][1]

            posterior_phi_t=sample_phi_posterior(Y=Y,
                                                 U=U,
                                                 X=X,
                                                 beta=posterior_beta_t,
                                                 sigma2=posterior_sigma2_t,
                                                 n_variables=1,
                                                 seed=None)
            
            posterior_z_t=gibbs_sampler_z_posterior(q=posterior_q_t, 
                                                    Y=Y,
                                                    U=U,
                                                    X=X,
                                                    phi=posterior_phi_t,
                                                    R2=posterior_R2_t,
                                                    n_iter=n_iter_gibbs_sampler_posterior_z,
                                                    n_variables=1,
                                                    seed=None)
            
            posterior_sigma2_t=sample_sigma2_posterior(Y=Y,
                                                       U=U,
                                                       X=X,
                                                       phi=posterior_phi_t,
                                                       R2=posterior_R2_t,
                                                       q=posterior_q_t,
                                                       Z=posterior_z_t,
                                                       n_variables=1,
                                                       seed=None)
            
            posterior_beta_tilde_t=sample_beta_tilde_posterior(Y=Y,
                                                               U=U,
                                                               X=X,
                                                               phi=posterior_phi_t,
                                                               R2=posterior_R2_t,
                                                               q=posterior_q_t,
                                                               sigma2=posterior_sigma2_t,
                                                               Z=posterior_z_t,
                                                               n_variables=1,
                                                               seed=None)
            
            posterior_beta_t=reconstruct_beta_from_beta_tilde_z(beta_tilde=posterior_beta_tilde_t, z=posterior_z_t, k=k)
            
            posterior_r2_q_theta_t=[posterior_R2_t, posterior_q_t, posterior_phi_t, posterior_beta_tilde_t, posterior_sigma2_t]

            list_r2_q_theta_posterior_var_n.append(posterior_r2_q_theta_t)
        
        list_r2_q_theta_posterior_var_n=list_r2_q_theta_posterior_var_n[burn_in_period:] #burn-in period
        dico_R2_q_theta_posterior["Posterior_"+str(n+1)]=list_r2_q_theta_posterior_var_n
    
    return dico_R2_q_theta_posterior

In [23]:
dico_theta_posterior_test=gibbs_sampler_theta_posterior(X=X,
                                                        Y=Y,
                                                        U=U,
                                                        a=a, b=b, A=A, B=B,
                                                        n_iter=110000,
                                                        n_iter_gibbs_sampler_posterior_z=1000,
                                                        burn_in_period=10000,
                                                        n_variables=1,
                                                        seed=seed)

ValueError: cannot reshape array of size 0 into shape (0)